<a href="https://colab.research.google.com/github/alfrepereyra/laboratorio/blob/main/spotify_laboratorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



necesito normalizar los datos de este csv y crear un nuevo dataframe, lo primero quitar todos los datos nulos o Nan y despues convertir todos los datos numericos a float entre 0 y 1 para poder trabajar mejor:



In [2]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

# Ruta del archivo CSV
archivo_csv = '/content/drive/MyDrive/UPSO/laboratorio_Spotify/Canciones_Spotify.csv'

# Cargar el DataFrame original desde el archivo CSV
df = pd.read_csv(archivo_csv)

# Eliminar datos nulos
df = df.dropna()

# Normalizar datos numéricos entre 0 y 1
scaler = MinMaxScaler()
columnas_numericas = df.select_dtypes(include=['float64']).columns
df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])

df_nuevo = pd.DataFrame(df)
print(df_nuevo)


      Unnamed: 0  acousticness  danceability  duration_ms    energy  \
0              0      0.010248      0.824826       204600  0.426363   
1              1      0.199998      0.720418       326933  0.350081   
2              2      0.034570      0.830626       185707  0.403987   
3              3      0.607034      0.431555       199413  0.328723   
4              4      0.180902      0.645012       392893  0.555533   
...          ...           ...           ...          ...       ...   
2012        2012      0.001062      0.535963       274404  0.932872   
2013        2013      0.088138      0.895592       182182  0.892189   
2014        2014      0.008610      0.597448       207200  0.935924   
2015        2015      0.001645      0.504640       185600  0.993897   
2016        2016      0.002821      0.375870       204520  0.915582   

      instrumentalness  key  liveness  loudness  mode  speechiness     tempo  \
0             0.022439    2  0.153862  0.741141     1     0.514441 

 divido el conjunto de datos normalizado en conjuntos de entrenamiento y prueba. quitando las columnas de artist y song title que puede dar problemas y aplico 4 modelos de machine learning

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

# Elimina la columna 'song_title' y 'artist' ya que no son numericos
X = df_nuevo.drop(['target', 'song_title',"artist"], axis=1)


y = df_nuevo['target']  # Etiquetas

# Divide el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. SVM (Support Vector Machines)
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
# Hacer predicciones en el conjunto de prueba
svm_predictions = svm_model.predict(X_test)
# Evaluar el rendimiento
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f'SVM Accuracy: {svm_accuracy}')

# 2. KNN (K-Nearest Neighbors)
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
# Hacer predicciones en el conjunto de prueba
knn_predictions = knn_model.predict(X_test)
# Evaluar el rendimiento
knn_accuracy = accuracy_score(y_test, knn_predictions)
print(f'KNN Accuracy: {knn_accuracy}')

# 3. Árbol de decisión
tree_model = DecisionTreeClassifier(max_depth=3)
tree_model.fit(X_train, y_train)
# Hacer predicciones en el conjunto de prueba
tree_predictions = tree_model.predict(X_test)
# Evaluar el rendimiento
tree_accuracy = accuracy_score(y_test, tree_predictions)
print(f'Decision Tree Accuracy: {tree_accuracy}')


#4 Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
# Hacer predicciones en el conjunto de prueba
nb_predictions = nb_model.predict(X_test)
# Evaluar el rendimiento
nb_accuracy = accuracy_score(y_test, nb_predictions)
print(f'Gaussian Naive Bayes Accuracy: {nb_accuracy}')


SVM Accuracy: 1.0
KNN Accuracy: 0.8910891089108911
Decision Tree Accuracy: 1.0
Gaussian Naive Bayes Accuracy: 0.9900990099009901


Realizar algún tipo de validación:
● Validación Simple
● Validación Cruzada k-fold

In [5]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Suponiendo que 'df_nuevo' contiene tus datos normalizados y 'target' es la columna de etiquetas
# y 'song_title', 'artist' son las columnas que deseas codificar
X = df_nuevo.drop(['target', 'song_title', 'artist'], axis=1)


y = df_nuevo['target']  # Etiquetas

# Divide el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos
models = {
    'SVM': SVC(kernel='linear'),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(max_depth=3),
    'Gaussian Naive Bayes': GaussianNB()
}

# Entrenamiento y evaluación
for model_name, model in models.items():
    # Entrenamiento
    model.fit(X_train, y_train)

    # Predicciones
    predictions = model.predict(X_test)

    # Evaluación
    accuracy = accuracy_score(y_test, predictions)
    print(f'{model_name} Accuracy: {accuracy}')

    # Validación Cruzada k-fold
    cv_scores = cross_val_score(model, X, y, cv=5)
    print(f'{model_name} Cross-Validation Scores: {cv_scores}')
    print(f'{model_name} Mean Cross-Validation Score: {cv_scores.mean()}\n')

SVM Accuracy: 1.0
SVM Cross-Validation Scores: [0.75990099 1.         1.         1.         0.74937965]
SVM Mean Cross-Validation Score: 0.9018561285408937

KNN Accuracy: 0.8910891089108911
KNN Cross-Validation Scores: [0.78960396 0.89356436 0.83126551 0.84615385 0.75186104]
KNN Mean Cross-Validation Score: 0.8224897427708033

Decision Tree Accuracy: 1.0
Decision Tree Cross-Validation Scores: [0.75247525 1.         1.         1.         0.74689826]
Decision Tree Mean Cross-Validation Score: 0.8998747021104094

Gaussian Naive Bayes Accuracy: 0.9900990099009901
Gaussian Naive Bayes Cross-Validation Scores: [0.78960396 1.         0.93548387 1.         0.77171216]
Gaussian Naive Bayes Mean Cross-Validation Score: 0.8993599980345429



**Evaluación y análisis del rendimiento de cada modelo:
● Matriz de confusión
● Precisión, recall y F1-score texto en negrita**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Función para mostrar la matriz de confusión en un mapa de calor
def mostrar_matriz_confusion(conf_matrix, title):
    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.xlabel('Predicho')
    plt.ylabel('Real')
    plt.show()

# Función para mostrar las métricas de clasificación
def mostrar_metricas_clasificacion(precision, recall, f1, title):
    labels = ['Precisión', 'Recall', 'F1-Score']
    values = [precision, recall, f1]

    plt.figure(figsize=(8, 4))
    bars = plt.bar(labels, values, color=['green', 'orange', 'blue'])
    plt.title(title)
    plt.ylabel('Puntuación')

    # Mostrar los valores de las barras
    for bar, value in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width() / 2 - 0.1, bar.get_height() + 0.02, f'{value:.2f}', fontsize=9)

    plt.show()

# Mostrar la matriz de confusión para SVM
mostrar_matriz_confusion(svm_conf_matrix, 'Matriz de Confusión SVM')
mostrar_metricas_clasificacion(svm_precision, svm_recall, svm_f1_score, 'Métricas de Clasificación SVM')

# Mostrar la matriz de confusión para KNN
mostrar_matriz_confusion(knn_conf_matrix, 'Matriz de Confusión KNN')
mostrar_metricas_clasificacion(knn_precision, knn_recall, knn_f1_score, 'Métricas de Clasificación KNN')

# Mostrar la matriz de confusión para Árbol de decisión
mostrar_matriz_confusion(tree_conf_matrix, 'Matriz de Confusión Árbol de decisión')
mostrar_metricas_clasificacion(tree_precision, tree_recall, tree_f1_score, 'Métricas de Clasificación Árbol de decisión')

# Mostrar la matriz de confusión para Naive Bayes
mostrar_matriz_confusion(nb_conf_matrix, 'Matriz de Confusión Naive Bayes')
mostrar_metricas_clasificacion(nb_precision, nb_recall, nb_f1_score, 'Métricas de Clasificación Naive Bayes')